<a href="https://colab.research.google.com/github/IsraelAbebe/Amharic-NLP/blob/master/Amharic-Word2Vec/Amharic_Skip_gram_Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Amharic Skip-gram Word2Vec
[Using SkipGram Udacity Tutorials.](https://github.com/udacity/deep-learning-v2-pytorch)

## 1. Loading Data

In [0]:
import re
from collections import Counter
import random
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
text_data_path = 'gdrive/My Drive/dataset/CACO/CACO_TEXT.txt'

with open(text_data_path) as f:
    text = f.read()

print(text[:300])

አሁን ወደ ዋናው የጉዞ ፕሮግራም እንመለስ ።
ደብዳቤውን ካነበብክ በኋላ " ጓደኛዬ እኮ እንዲህ አለኝ ! " ብለህ በደስታ ለሌሎች ትናገር ይሆናል ።
የትምህርት ቤቶች እገዛና ድጋፍ ከፍተኛ መሆን እንዳለበትም አስተያየት ሰጥተዋል ።
አብደሻል ?
ደንብ ካልተከበረ የአዲስ አበባ ከተማን ዕድገት ማሳለጥም አይቻልም ።
የመማጸኛ ከተሞች ( 9 34 )
ከዚህ በተጨማሪም ፣ የእርሻ ስራ ከጀመሩ ወዲህ በማኅበር ተደራጅተው 100ሺ ብር ብድር ወስደው በገቢ ማስገኛ ስራዎች ላይ ተሰማር


## 2.Pre-processing

In [0]:
def preprocess(text):
    # Replace punctuation with tokens so we can use them in our model
    text = text.lower()
    text = text.replace('.', ' <PERIOD> ')
    text = text.replace(',', ' <COMMA>')
    text = text.replace('"', ' <QUOTATION_MARK> ')
    text = text.replace(';', ' <SEMICOLON> ')
    text = text.replace('!', ' <EXCLAMATION_MARK> ')
    text = text.replace('?', ' <QUESTION_MARK> ')
    text = text.replace('(', ' <LEFT_PAREN> ')
    text = text.replace(')', ' <RIGHT_PAREN> ')
    text = text.replace('--', '<HYPHENS>')
    text = text.replace('?', ' <QUESTION_MARK> ')
    text = text.replace('።', ' <PERIOD> ')
    text = text.replace('፣', ' <PERIOD> ')
    # text = text.replace('\n', ' <NEW_LINE> ')
    text = text.replace(':', '')
    text = text.replace('፤',' <SEMICOLON> ')
    text = text.replace('…',' <HYPHENS> ')
    text = text.replace('[',' <SQUARE_LEFT_PAREN> ')
    text = text.replace(']',' <SQUARE_RIGHT_PAREN> ')
    words = text.split()
    
    # Remove all words with  5 or fewer occurences
    word_counts = Counter(words)
    trimmed_words = [word for word in words if word_counts[word] > 5]

    return trimmed_words

In [0]:
words = preprocess(text)
print(words[:500])

['አሁን', 'ወደ', 'ዋናው', 'የጉዞ', 'ፕሮግራም', 'እንመለስ', '<PERIOD>', 'ደብዳቤውን', 'ካነበብክ', 'በኋላ', '<QUOTATION_MARK>', 'ጓደኛዬ', 'እኮ', 'እንዲህ', 'አለኝ', '<EXCLAMATION_MARK>', '<QUOTATION_MARK>', 'ብለህ', 'በደስታ', 'ለሌሎች', 'ትናገር', 'ይሆናል', '<PERIOD>', 'የትምህርት', 'ቤቶች', 'እገዛና', 'ድጋፍ', 'ከፍተኛ', 'መሆን', 'እንዳለበትም', 'አስተያየት', 'ሰጥተዋል', '<PERIOD>', 'አብደሻል', '<QUESTION_MARK>', 'ደንብ', 'ካልተከበረ', 'የአዲስ', 'አበባ', 'ከተማን', 'ዕድገት', 'አይቻልም', '<PERIOD>', 'የመማጸኛ', 'ከተሞች', '<LEFT_PAREN>', '9', '34', '<RIGHT_PAREN>', 'ከዚህ', 'በተጨማሪም', '<PERIOD>', 'የእርሻ', 'ስራ', 'ከጀመሩ', 'ወዲህ', 'በማኅበር', 'ተደራጅተው', '100ሺ', 'ብር', 'ብድር', 'ወስደው', 'በገቢ', 'ማስገኛ', 'ስራዎች', 'ላይ', 'ተሰማርተዋል', '<PERIOD>', 'ባለ', 'ድርሻ', 'አካላትንና', 'የክልሉን', 'ማህበረሰብ', 'ፍላጎት', 'መሰረት', 'ያደረጉ', 'የምርምር', 'ፕሮፖዛሎች', 'የዩኒቨርሲቲውን', 'ዌብ', 'ሳይት', 'በመጠቀም', 'በተደረገው', 'የፕሮፖዛል', 'ጥሪ', 'መሰረት', 'በአጠቃላይ', '85', 'የምርምር', 'ፕሮፖዛሎች', 'ቀርበዋል', '<PERIOD>', 'ከተስፋ', 'አስቆራጩ', 'የኤምባሲው', 'ምላሽ', 'በኋላም', 'የሚመጣውን', 'በፀጋ', 'ለመቀበል', 'ወስነው', 'ጥቅምት', 'ሃያ', 'አራትን', 'መጠባበቅ', 'ጀመሩ', '<PERIOD>', 'ሌላው', 'የአከባቢው', 'ነዋሪ', 'አርሶ', 'አ

In [0]:
print("Total words in text: {}".format(len(words)))
print("Unique words: {}".format(len(set(words))))

Total words in text: 20754429
Unique words: 172040


## 3.Dictionaries

In [0]:
def create_lookup_tables(words):
    """
    Create lookup tables for vocabulary
    :param words: Input list of words
    :return: Two dictionaries, vocab_to_int, int_to_vocab
    """
    word_counts = Counter(words)
    # sorting the words from most to least frequent in text occurrence
    sorted_vocab = sorted(word_counts, key=word_counts.get, reverse=True)
    # create int_to_vocab dictionaries
    int_to_vocab = {ii: word for ii, word in enumerate(sorted_vocab)}
    vocab_to_int = {word: ii for ii, word in int_to_vocab.items()}

    return vocab_to_int, int_to_vocab

In [0]:
vocab_to_int, int_to_vocab = create_lookup_tables(words)
int_words = [vocab_to_int[word] for word in words]

print(int_words[:30])

[44, 6, 663, 4454, 593, 13203, 0, 12123, 102661, 33, 1, 4995, 277, 45, 600, 15, 1, 1373, 1877, 940, 38686, 95, 0, 292, 188, 20812, 144, 47, 114, 5905]


### 3.Subsampling

In [0]:
threshold = 1e-5
word_counts = Counter(int_words)
print(list(word_counts.items())[0])  # dictionary of int_words, how many times they appear

total_count = len(int_words)
freqs = {word: count/total_count for word, count in word_counts.items()}
p_drop = {word: 1 - np.sqrt(threshold/freqs[word]) for word in word_counts}
# discard some frequent words, according to the subsampling equation
# create a new list of words for training
train_words = [word for word in int_words if random.random() < (1 - p_drop[word])]

print(train_words[:30])

(44, 25669)
[593, 13203, 12123, 102661, 600, 38686, 20812, 144, 5905, 138671, 76062, 5563, 966, 30238, 5173, 17095, 23156, 1092, 3254, 9833, 681, 16212, 61388, 4853, 948, 49342, 21320, 24344, 13141, 671]


### 3.Making batches

In [0]:
def get_target(words, idx, window_size=5):
    ''' Get a list of words in a window around an index. '''
    
    R = np.random.randint(1, window_size+1)
    start = idx - R if (idx - R) > 0 else 0
    stop = idx + R
    target_words = words[start:idx] + words[idx+1:stop+1]
    
    return list(target_words)

In [0]:
# test your code!

# run this cell multiple times to check for random window selection
int_text = [i for i in range(10)]
print('Input: ', int_text)
idx=5 # word index of interest

target = get_target(int_text, idx=idx, window_size=5)
print('Target: ', target)

Input:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Target:  [2, 3, 4, 6, 7, 8]


In [0]:
def get_batches(words, batch_size, window_size=5):
    ''' Create a generator of word batches as a tuple (inputs, targets) '''
    
    n_batches = len(words)//batch_size
    
    # only full batches
    words = words[:n_batches*batch_size]
    
    for idx in range(0, len(words), batch_size):
        x, y = [], []
        batch = words[idx:idx+batch_size]
        for ii in range(len(batch)):
            batch_x = batch[ii]
            batch_y = get_target(batch, ii, window_size)
            y.extend(batch_y)
            x.extend([batch_x]*len(batch_y))
        yield x, y

In [0]:
int_text = [i for i in range(20)]
x,y = next(get_batches(int_text, batch_size=4, window_size=5))

print('x\n', x)
print('y\n', y)

x
 [0, 0, 0, 1, 1, 1, 2, 2, 3, 3, 3]
y
 [1, 2, 3, 0, 2, 3, 1, 3, 0, 1, 2]


In [0]:
def cosine_similarity(embedding, valid_size=16, valid_window=100, device='cpu'):
    """ Returns the cosine similarity of validation words with words in the embedding matrix.
        Here, embedding should be a PyTorch embedding module.
    """
    
    # Here we're calculating the cosine similarity between some random words and 
    # our embedding vectors. With the similarities, we can look at what words are
    # close to our random words.
    
    # sim = (a . b) / |a||b|
    
    embed_vectors = embedding.weight
    
    # magnitude of embedding vectors, |b|
    magnitudes = embed_vectors.pow(2).sum(dim=1).sqrt().unsqueeze(0)
    
    # pick N words from our ranges (0,window) and (1000,1000+window). lower id implies more frequent 
    valid_examples = np.array(random.sample(range(valid_window), valid_size//2))
    valid_examples = np.append(valid_examples,
                               random.sample(range(1000,1000+valid_window), valid_size//2))
    valid_examples = torch.LongTensor(valid_examples).to(device)
    
    valid_vectors = embedding(valid_examples)
    similarities = torch.mm(valid_vectors, embed_vectors.t())/magnitudes
        
    return valid_examples, similarities

## 4.SkipGram model

In [0]:
import torch
from torch import nn
import torch.optim as optim

In [0]:
class SkipGram(nn.Module):
    def __init__(self, n_vocab, n_embed):
        super().__init__()
        
        self.embed = nn.Embedding(n_vocab, n_embed)
        self.output = nn.Linear(n_embed, n_vocab)
        self.log_softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        x = self.embed(x)
        scores = self.output(x)
        log_ps = self.log_softmax(scores)
        
        return log_ps

In [0]:
# check if GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'

embedding_dim=300 # you can change, if you want

model = SkipGram(len(vocab_to_int), embedding_dim).to(device)
criterion = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

print_every = 1000
steps = 0
epochs = 5

# train for some number of epochs
for e in range(epochs):
    
    # get input and target batches
    for inputs, targets in get_batches(train_words, 128):
        steps += 1
        inputs, targets = torch.LongTensor(inputs), torch.LongTensor(targets)
        inputs, targets = inputs.to(device), targets.to(device)
        Untitled4.ipynbUntitled4.ipynb
        log_ps = model(inputs)
        loss = criterion(log_psUntitled4.ipynb, targets)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if steps % print_every == 0:                  
            # getting examples and similarities      
            valid_examples, valid_similarities = cosine_similarity(model.embed, device=device)
            _, closest_idxs = valid_similarities.topk(6) # topk highest similarities
            
            valid_examples, closest_idxs = valid_examples.to('cpu'), closest_idxs.to('cpu')
            for ii, valid_idx in enumerate(valid_examples):
                closest_words = [int_to_vocab[idx.item()] for idx in closest_idxs[ii]][1:]
                print(int_to_vocab[valid_idx.item()] + " | " + ', '.join(closest_words))
            print("...")

አለ | ንግግሯ, የእማማ, ቀልብና, የጻድቁን, እንደማይመጡ
<RIGHT_PAREN> | ከችግኝ, አሽከሩ, መጣችሁ, መመሪያዎች, ከበታች
ጉዳይ | አረጋገጠልኝ, ሸልሟል, ከብዛቱ, እንደሚድን, አውግቸው
ነበር | መሆኑንም, ትንግርታዊ, ውበቱ, ከተያያዙ, አደራጅቶ
ኢትዮጵያ | ባጋጠሙን, መፅሐፍትን, ለመውጋት, በምርት, ርቀቱ
ይህን | ሕልማቸው, ይጀመር, የዳኝነቱ, ለኑኃሚን, እንዲያ
መንግሥት | ተምሳሌት, አዕምሮውን, ጡሯ, ሸሽተው, የፈረሰኛ
ችግር | መነጣጠል, ንቅሳት, ibex, በፋራን, አዘጋጅቻለሁ
ያመለክታል | ግንባሮች, ካልተሳሳትኩ, ሳያረጋግጡ, የተደነቁ, ኤርትራውያንን
መቼ | በተሰጠበት, የአቤቱታው, አሰብኩኝ, ይግለጥልህ, የሚገጥሟቸውን
ከተለያዩ | እናልፈዋለን, ማርገዝ, ጥያቄው, በጥንቃቄ, መጐዳት
በውጭ | ያበረከተውን, ይመልስለታል, ከሥጋዊ, የሚደፈር, ያሳተፉ
ያደረገ | በጭንቅላታቸው, ለመደጋገፍ, የማይታበል, ፓርቲያቸውን, የሚያጠናክርበት
ይሰጣል | ባለመሆናቸውም, መመዘኛዎችና, የ11ኛው, ስለሰጠው, የበቃበት
አይ | እየመራ, ጣመን, እዬዞሩ, ጓደኞቻቸው, ከ1935
አንደኛው | ፈራሽ, እንዳይጠቀስ, የህሙማኑን, በዝማሬ, ተርጓሚና
...
' | የኩርዲስታን, ስለሀገራችን, መያዙም, ተቀባዩ, የሚለገሰው
ነገር | የሚጎርፉት, ልባሽ, ተሰርቶ, አዟቸዋል, ትሆናላችሁ
ሆኖ | እየተመለስን, ካሳቫ, ከመሪ, ከመመገብ, ስላልተፈቀደላቸው
ዘመን | እየተጎተተ, አዋቂው, ማነቆ, ያልቻለችው, የሚሆኑትም
<QUESTION_MARK> | አለመናገሩ, የሥልጠናው, የቱም, ዕንቁራሪቱ, ያዘጋጅላችኋል
ይችላል | እንክርዳዱ, ነጥቦችህን, ይሰቃይ, በሚፈጠሩ, የሚወጡና
እኔ | በማጽናናት, ገዥ, ከማቀድ, ርህራሄና, ለእህቶቻቸው
ምክንያት | እያመጣ, የሚገድለው, በወ

## 3.Visualizing the word vectors

In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

In [0]:
viz_words = 600
tsne = TSNE()
embed_tsne = tsne.fit_transform(embeddings[:viz_words, :])

In [0]:
fig, ax = plt.subplots(figsize=(16, 16))
for idx in range(viz_words):
    plt.scatter(*embed_tsne[idx, :], color='steelblue')
    plt.annotate(int_to_vocab[idx], (embed_tsne[idx, 0], embed_tsne[idx, 1]), alpha=0.7)